In [298]:
import gensim
import numpy as np

In [299]:
import numpy
import sys
import scipy.stats

def evaluate(pred,gold):
    
    f=open(pred, "rb")
    pred_lines=f.readlines()
    f.close()
    
    f=open(gold, "rb")
    gold_lines=f.readlines()
    f.close()
    

    if(len(pred_lines)==len(gold_lines)):       
        # align tweets ids with gold scores and predictions
        data_dic={}
        
        for line in gold_lines:
            line = line.decode()
            parts=line.split('\t')
            if len(parts)==4:   
                data_dic[int(parts[0])]=[float(line.split('\t')[3])]
            else:
                raise ValueError('Format problem.')
        
        
        for line in pred_lines:
            line = line.decode()
            parts=line.split('\t')
            if len(parts)==4:  
                if int(parts[0]) in data_dic:
                    try:
                        data_dic[int(parts[0])].append(float(line.split('\t')[3]))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[int(parts[0])].append(0.5)
                else:
                    raise ValueError('Invalid tweet id.')
            else:
                raise ValueError('Format problem.')
            
            
        
        # lists storing gold and prediction scores
        gold_scores=[]  
        pred_scores=[]
         
        
        # lists storing gold and prediction scores where gold score >= 0.5
        gold_scores_range_05_1=[]
        pred_scores_range_05_1=[]
         
            
        for id in data_dic:
            if(len(data_dic[id])==2):
                gold_scores.append(data_dic[id][0])
                pred_scores.append(data_dic[id][1])
                if(data_dic[id][0]>=0.5):
                    gold_scores_range_05_1.append(data_dic[id][0])
                    pred_scores_range_05_1.append(data_dic[id][1])
            else:
                raise ValueError('Repeated id in test data.')
                
      
        # return zero correlation if predictions are constant
        if numpy.std(pred_scores)==0 or numpy.std(gold_scores)==0:
            return (0,0,0,0)
        

        pears_corr=scipy.stats.pearsonr(pred_scores,gold_scores)[0]                                    
        spear_corr=scipy.stats.spearmanr(pred_scores,gold_scores)[0]   


        pears_corr_range_05_1=scipy.stats.pearsonr(pred_scores_range_05_1,gold_scores_range_05_1)[0]                                    
        spear_corr_range_05_1=scipy.stats.spearmanr(pred_scores_range_05_1,gold_scores_range_05_1)[0]           
        
      
        return (pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1)
                                           
                          
        
    else:
        raise ValueError('Predictions and gold data have different number of lines.')

# Load pre-trained word vectors

In [2]:
import gc
gc.collect()

0

In [3]:
word_vector_path = "/home/v2john/"
wassa_home = "/home/v2john/WASSA-Task/"

In [5]:
# Google news pretrained vectors
wv_model_path = word_vector_path + "GoogleNews-vectors-negative300.bin.gz"
wv_model = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path, binary=True, unicode_errors='ignore')

In [42]:
# Twitter pretrained vectors
wv_model_path_1 = word_vector_path + "word2vec_twitter_model.bin"
wv_model_1 = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path_1, binary=True, unicode_errors='ignore')

In [407]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = np.array(embedding)
    print("Done.",len(model)," words loaded!")
    return model

wv_model_path_2 = word_vector_path + "glove.twitter.27B.200d.txt"
wv_model_2 = loadGloveModel(wv_model_path_2)

Loading Glove Model
Done. 1193514  words loaded!


In [408]:
w2v_dimensions = len(wv_model['word'])
w2v_dimensions_1 = len(wv_model_1['word'])
w2v_dimensions_2 = len(wv_model_2['word'])
print(w2v_dimensions, w2v_dimensions_1, w2v_dimensions_2)

300 400 200


In [409]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

lmtzr = WordNetLemmatizer()

def remove_stopwords(string):
    split_string = \
        [word for word in string.split()
         if word not in stopwords.words('english')]
    
    return " ".join(split_string)

In [410]:
import re

def clean_str(string):
    string = re.sub(r"@[A-Za-z0-9_s(),!?\'\`]+", "", string) # removing any twitter handle mentions
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)

    return remove_stopwords(string.strip().lower())

# Load affect intensity lexica

In [411]:
affect_intensity_file_path = \
    wassa_home + \
    "lexicons/NRC-AffectIntensity-Lexicon.txt"

def get_word_affect_intensity_dict(emotion):
    word_intensities = dict()

    with open(affect_intensity_file_path) as affect_intensity_file:
        for line in affect_intensity_file:
            word_int_array = line.replace("\n", "").split("\t")

            if (word_int_array[2] == emotion):
                word_intensities[word_int_array[0]] = float(word_int_array[1])

    return word_intensities

In [412]:
sentiment_emotion_lex_file_path = \
    wassa_home + \
    "lexicons/NRC-Sentiment-Emotion-Lexicons/Lexicons/NRC-Emotion-Lexicon-v0.92/" + \
    "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

def get_affect_presence_list(emotion):
    word_list = list()
    
    with open(sentiment_emotion_lex_file_path) as sentiment_emotion_lex_file:
        for line in sentiment_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")

            if (word_array[1] == emotion and word_array[2] == '1'):
                word_list.append(word_array[0])
                
    return word_list

In [413]:
hashtag_emotion_lex_file_path = \
    wassa_home + \
    "lexicons/NRC-Sentiment-Emotion-Lexicons/Lexicons/NRC-Hashtag-Emotion-Lexicon-v0.2/" + \
    "NRC-Hashtag-Emotion-Lexicon-v0.2.txt"
    
def get_hashtag_emotion_intensity(emotion):
    hastag_intensities = dict()
    
    with open(hashtag_emotion_lex_file_path) as hashtag_emotion_lex_file:
        for line in hashtag_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")

            if (word_array[0] == emotion):
                hastag_intensities[word_array[1].replace("#", "")] = float(word_array[2])
                
    return hastag_intensities

In [414]:
import re
pattern = re.compile("maths exam .+ have")
print(pattern.match("math exam fhrebvrn have"))

None


In [415]:
emoticon_lexicon_file_path = \
    wassa_home + \
    "lexicons/NRC-Sentiment-Emotion-Lexicons/Lexicons/NRC-Hashtag-Emotion-Lexicon-v0.2/" + \
    "NRC-Hashtag-Emotion-Lexicon-v0.2.txt"

In [416]:
print(clean_str("A night where depression is winning... #depression #fml #help"))

night depression winning depression fml help


In [417]:
class Tweet(object):

    def __init__(self, id, text, emotion, intensity):
        self.id = id
        self.text = text
        self.emotion = emotion
        self.intensity = intensity

    def __repr__(self):
        return \
            "id: " + self.id + \
            ", text: " + self.text + \
            ", emotion: " + self.emotion + \
            ", intensity: " + self.intensity

In [418]:
def read_training_data(training_data_file_path):

    train_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            train_list.append(Tweet(array[0], clean_str(array[1]), array[2], float(array[3])))
    return train_list
            
def read_test_data(training_data_file_path):

    test_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            test_list.append(Tweet(array[0], clean_str(array[1]), array[2], None))
    return test_list

In [468]:
emotion = "joy"

In [469]:
training_data_file_path = \
    wassa_home + "dataset/" + \
    emotion + "-ratings-0to1.train.txt"
test_data_file_path = \
    wassa_home + "dataset/" + \
    emotion + "-ratings-0to1.dev.target.txt"
predictions_file_path = \
    wassa_home + "predictions/" + \
    emotion + "-pred.txt"
gold_set_path = \
    wassa_home + "dataset/gold-set/" + \
    emotion + "-ratings-0to1.dev.gold.txt"

In [470]:
training_tweets = read_training_data(training_data_file_path)

score_train = list()
tweet_train = list()
for tweet in training_tweets:
    tweet_train.append(tweet.text)
    score_train.append(float(tweet.intensity))
print(len(score_train))

823


In [471]:
from sklearn.preprocessing import PolynomialFeatures

poly_emo_int = PolynomialFeatures(10)
word_intensities = get_word_affect_intensity_dict(emotion)

def get_emo_int_vector(tweet):
    score = 0.0
    for word in word_intensities.keys():
        if word in tweet:
            score += tweet.count(word) * float(word_intensities[word])
    
    return poly_emo_int.fit_transform(np.array([score]).reshape(1, -1))[0].tolist()

In [472]:
def get_word2vec_embedding(tweet, model, dimensions):
    vector_list = list()
    for word in tweet.split():
        try:
            vector_list.append(model[word])
        except Exception as e:
            pass

    if len(vector_list) == 0:
        vec_rep = np.zeros(dimensions).tolist()
    else:
        try:
            vec_rep = sum(vector_list) / float(len(vector_list))
        except Exception as e:
            print(vector_list)
            print(e)
            raise Exception

    return vec_rep

In [473]:
word_list = get_affect_presence_list(emotion)

def get_sentiment_emotion_feature(tweet):
    for word in word_list:
        if word in tweet.split():
            return [1.0]
    
    return [0.0]

In [474]:
poly_emo_int = PolynomialFeatures(10)
hashtag_emotion_intensities = get_hashtag_emotion_intensity(emotion)

def get_hashtag_emotion_vector(tweet):
    score = 0.0
    for word in hashtag_emotion_intensities.keys():
        if word in tweet:
            score += tweet.count(word) * float(hashtag_emotion_intensities[word])
    
    return poly_emo_int.fit_transform(np.array([score]).reshape(1, -1))[0].tolist()

In [475]:
def vectorize_tweets(tweet_list):
    vectors = list()

    for tweet in tweet_list:
        x_vector = list()
        x_vector.extend(get_emo_int_vector(tweet))
#         x_vector.extend(get_word2vec_embedding(tweet, wv_model, w2v_dimensions))
#         x_vector.extend(get_word2vec_embedding(tweet, wv_model_1, w2v_dimensions_1))
        x_vector.extend(get_word2vec_embedding(tweet, wv_model_2, w2v_dimensions_2))
        x_vector.extend(get_sentiment_emotion_feature(tweet))
        x_vector.extend(get_hashtag_emotion_vector(tweet))
        vectors.append(x_vector)
    
    return vectors

In [476]:
x_train = vectorize_tweets(tweet_train)
print(len(x_train))
dimension = len(x_train[0])
print(dimension)

# with open("/tmp/dump.txt", 'w') as dump_file:
#     for i in range(len(x_train)):
#         if dimension != len(x_train[i]):
#             print(len(x_train[i]), i)
#         dump_file.write(str(x_train[i]))
#         dump_file.write("\n")

823
223


In [477]:
test_tweets = read_test_data(test_data_file_path)
tweet_test = list()
for tweet in test_tweets:
    tweet_test.append(tweet.text)

x_test = vectorize_tweets(tweet_test)
print(len(x_test))

79


In [478]:
from sklearn import ensemble, svm
from xgboost import XGBRegressor

ml_model = ensemble.GradientBoostingRegressor(max_depth=3, n_estimators=100)
# ml_model = XGBRegressor(n_estimators=100)
ml_model.fit(x_train, score_train)

y_test = ml_model.predict(x_test)

In [479]:
y_gold = read_training_data(gold_set_path)

data_dict = dict()
diff = 0
for i in range(len(y_gold)):
    if y_gold[i].intensity >= 0.5:
        diff += y_gold[i].intensity - y_test[i]
#         print([tweet_test[i], str(y_test[i]), str(y_gold[i].intensity)])
print(diff/len(y_gold))

0.053384242218


In [480]:
with open(predictions_file_path, 'w') as predictions_file:
    for i in range(len(y_test)):
        predictions_file.write(
            str(test_tweets[i].id) + "\t" + test_tweets[i].text + "\t" +
            test_tweets[i].emotion +"\t" + str(y_test[i]) + "\n"
        )

# Evaluation based on Pearson and Spearman co-efficients

In [481]:
print(emotion)
print("pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1")
print(evaluate(predictions_file_path, gold_set_path))

joy
pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
(0.74906004238663104, 0.75173326247046957, 0.57200630132669938, 0.56193354664557726)


# Overall Score estimation

In [482]:
pears_corr_sum = 0
spear_corr_sum = 0
pears_corr_range_05_1_sum = 0
spear_corr_range_05_1_sum = 0

for emotion in ['anger', 'fear', 'sadness', 'joy']:
    print("\n### " + emotion)
    predictions_file_path = \
        wassa_home + "predictions/" + \
        emotion + "-pred.txt"
    gold_set_path = \
        wassa_home + "dataset/gold-set/" + \
        emotion + "-ratings-0to1.dev.gold.txt"
    print("| pears-corr | spear-corr | pears-corr-range-05-1 | spear-corr-range-05-1 |")
    print("| --- | --- | --- | --- |")
    pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1 = \
        evaluate(predictions_file_path, gold_set_path)
    print("| " + str(pears_corr) + " | " + str(spear_corr) + " | " + \
          str(pears_corr_range_05_1) + " | " + str(spear_corr_range_05_1) + " |")
    pears_corr_sum += pears_corr
    spear_corr_sum += spear_corr
    pears_corr_range_05_1_sum += pears_corr_range_05_1
    spear_corr_range_05_1_sum += spear_corr_range_05_1
    
print("\n### Average Scores")
print("| pears-corr | spear-corr | pears-corr-range-05-1 | spear-corr-range-05-1 |")
print("| --- | --- | --- | --- |")
print("| " + str(pears_corr_sum/4) + " | " + str(spear_corr_sum/4) + " | " + \
      str(pears_corr_range_05_1_sum/4) + " | " + str(spear_corr_range_05_1_sum/4) + " |")


### anger
| pears-corr | spear-corr | pears-corr-range-05-1 | spear-corr-range-05-1 |
| --- | --- | --- | --- |
| 0.612036860479 | 0.552446724054 | 0.467200029867 | 0.354802970695 |

### fear
| pears-corr | spear-corr | pears-corr-range-05-1 | spear-corr-range-05-1 |
| --- | --- | --- | --- |
| 0.617835476757 | 0.598443840634 | 0.539385152594 | 0.525558970829 |

### sadness
| pears-corr | spear-corr | pears-corr-range-05-1 | spear-corr-range-05-1 |
| --- | --- | --- | --- |
| 0.480054127113 | 0.473336454883 | 0.342865345145 | 0.242082646714 |

### joy
| pears-corr | spear-corr | pears-corr-range-05-1 | spear-corr-range-05-1 |
| --- | --- | --- | --- |
| 0.749060042387 | 0.75173326247 | 0.572006301327 | 0.561933546646 |

### Average Scores
| pears-corr | spear-corr | pears-corr-range-05-1 | spear-corr-range-05-1 |
| --- | --- | --- | --- |
| 0.614746626684 | 0.59399007051 | 0.480364207233 | 0.421094533721 |


# Simple Neural Network Implementation in Keras

In [29]:
import theano

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# define base model
_, dim_size = (np.array(x_train).shape)
print(dim_size)
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(500, activation='relu', input_dim=dim_size))
    model.add(Dropout(0.25))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(33, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=1000, batch_size=5, verbose=0)
estimator.fit(x_train, score_train)

In [ ]:
y_test = estimator.predict(x_test)
print(len(y_test))

In [ ]:
with open(predictions_file_path, 'w') as predictions_file:
    for i in range(len(y_test)):
        predictions_file.write(
            str(test_tweets[i].id) + "\t" + test_tweets[i].text + "\t" +
            test_tweets[i].emotion +"\t" + str(y_test[i]) + "\n"
        )